In [42]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest

from create_train_test_val_maps import *

import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
selected_codes = [0, 45021, 44004, 43004, 45008, 45002, 45007]

In [164]:
# get global index from (row, col) index
def sub2ind(array_shape, row, col):
    ind = row*array_shape[1] + col
    if row < 0 or row >= array_shape[0]:
        ind = -1
    if col < 0 or col >= array_shape[1]:
        ind = -1
    return ind

# get (row, col) index from global index
def ind2sub(array_shape, ind):
    row = int(ind) / array_shape[1]
    col = ind % array_shape[1]
    if ind < 0:
        row = -1
        col = -1
    if ind >=  array_shape[0]*array_shape[1]:
        row = -1
        col = -1
    return (row, col)

def one_hot_labels(labels, num_classes):
    one_hot_labels = np.zeros((labels.size, num_classes))
    one_hot_labels[np.arange(labels.size),labels.astype(int)] = 1
    return one_hot_labels

def get_data(selected_codes, data_map, num_windows, window_size):
    X = []
    Y = []
    for vehicleID in data_map.keys():
        for ATA6code in data_map[vehicleID].keys():
            if ATA6code not in selected_codes:
                continue
            for window in data_map[vehicleID][ATA6code].keys():
                for sequence in data_map[vehicleID][ATA6code][window]:
                    Y.extend(list(np.ones((window_size))*sub2ind((len(selected_codes),num_windows),selected_codes.index(ATA6code), window)))
                    X.extend(sequence.as_matrix())
    X = np.array(X).astype(float)
    Y = np.array(Y).astype(int)
    return X, Y

def save_file(filename, m):
    with open(filename+'.pkl', 'wb') as handle:
        pickle.dump(m, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def open_file(filename):
    m = None
    with open(filename+'.pkl', 'rb') as handle:
        m = pickle.load(handle)
    return m

In [167]:
num_epochs = 2
batch_size = 100
num_classes = 70
num_features = 61
num_trees = 70
max_nodes = 10000

data_sizes = [(5,50),(5,20), (10,10), (5,100), (1,50)]
trees = [70]

train_acc_all = {}
val_acc_all = {}
train_loss_all = {}

for num_windows, window_size in data_sizes:
    train_combined, val_combined, test_combined = load_maps(num_windows, window_size)

    train_acc_all[(num_windows,window_size)] = {}
    val_acc_all[(num_windows,window_size)] = {}
    train_loss_all[(num_windows,window_size)] = {}
    
    train_acc_trees = train_acc_all[(num_windows,window_size)]
    train_loss_trees = train_loss_all[(num_windows,window_size)]
    val_acc_trees = val_acc_all[(num_windows,window_size)]
    
    X_train, labels_train = get_data(selected_codes, train_combined, num_windows, window_size)
    X_val, labels_val = get_data(selected_codes, val_combined, num_windows, window_size)
    
    num_iters = max(X_train.shape[0]//batch_size,1)
    
    for num_trees in trees:
        print("Training tree {} with num {} and size {}".format(num_trees, num_windows, window_size))

        train_acc_trees[num_trees] = []
        train_loss_trees[num_trees] = []

        train_acc = train_acc_trees[num_trees]
        train_loss = train_loss_trees[num_trees]

        tf.reset_default_graph()

        # Input and Target data
        X = tf.placeholder(tf.float32, shape=[None, num_features])
        # For random forest, labels must be integers (the class id)
        Y = tf.placeholder(tf.int32, shape=[None])

        # Random Forest Parameters
        hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                              num_features=num_features,
                                              num_trees=num_trees,
                                              max_nodes=max_nodes).fill()

        # Build the Random Forest
        forest_graph = tensor_forest.RandomForestGraphs(hparams)
        # Get training graph and loss
        train_op = forest_graph.training_graph(X, Y)
        loss_op = forest_graph.training_loss(X, Y)

        # Measure the accuracy
        infer_op = forest_graph.inference_graph(X)
        correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
        accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        # Initialize the variables (i.e. assign their default value)
        init_vars = tf.global_variables_initializer()

        # Start TensorFlow session
        sess = tf.Session()

        # Run the initializer
        sess.run(init_vars)

        # Training
        for e in range(num_epochs):
            for i in range(num_iters):
                start = i*batch_size
                end = (i+1)*batch_size
                batch_x = X_train[start:end]
                batch_y = labels_train[start:end]
                _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
                if i % 100 == 0:
                    acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
                    train_acc.append([i,acc])
                    train_loss.append([i,l])
                    #print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

        val_acc_trees[num_trees] = sess.run(accuracy_op, feed_dict={X: X_val, Y: labels_val})

Training tree 70 with num 5 and size 50
INFO:tensorflow:Constructing forest with params = 


INFO:tensorflow:Constructing forest with params = 


INFO:tensorflow:{'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'num_classes': 70, 'num_outputs': 1, 'num_features': 61, 'split_initializations_per_input': 1, 'min_split_samples': 5, 'base_random_seed': 0, 'bagged_num_features': 61, 'regression': False, 'num_trees': 70, 'num_output_columns': 71, 'max_fertile_nodes': 5000, 'split_after_samples': 250, 'dominate_fraction': 0.99, 'num_splits_to_consider': 61, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'max_nodes': 10000, 'bagged_features': None}


INFO:tensorflow:{'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'num_classes': 70, 'num_outputs': 1, 'num_features': 61, 'split_initializations_per_input': 1, 'min_split_samples': 5, 'base_random_seed': 0, 'bagged_num_features': 61, 'regression': False, 'num_trees': 70, 'num_output_columns': 71, 'max_fertile_nodes': 5000, 'split_after_samples': 250, 'dominate_fraction': 0.99, 'num_splits_to_consider': 61, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'max_nodes': 10000, 'bagged_features': None}


INFO:tensorflow:training graph for tree: 0


...

INFO:tensorflow:training graph for tree: 67


INFO:tensorflow:training graph for tree: 68


INFO:tensorflow:training graph for tree: 68


INFO:tensorflow:training graph for tree: 69


INFO:tensorflow:training graph for tree: 69


In [154]:
save_file('train_acc_all_last0',train_acc_all)
save_file('val_acc_all_last0',val_acc_all)
save_file('train_loss_all_last0',train_loss_all)

In [168]:
val_acc_all

{(1, 50): {70: 0.72350007},
 (5, 20): {70: 0.51269031},
 (5, 50): {70: 0.55261898},
 (5, 100): {70: 0.59319448},
 (10, 10): {70: 0.48866507}}

In [59]:
val_acc_all

{(5, 20): {70: 0.18717949},
 (5, 50): {70: 0.15520833},
 (5, 100): {70: 0.17974025}}

In [13]:
save_file('train_acc_all',train_acc_all)
save_file('val_acc_all',val_acc_all)
save_file('train_loss_all',train_loss_all)

In [16]:
m = open_file('val_acc_all')

In [17]:
m

{(5, 20): {10: 0.5071066, 70: 0.52487302, 100: 0.52055836, 200: 0.52182734},
 (5, 50): {10: 0.53309518, 70: 0.54630941, 100: 0.53547609, 200: 0.5424999},
 (10, 10): {10: 0.48639807, 70: 0.48589429, 100: 0.49244341, 200: 0.49093208}}

In [139]:
X_train.shape

(0,)

In [175]:
num_windows, window_size = (2,50)
train_combined, val_combined, test_combined = load_maps(num_windows, window_size)
X_train, labels_train = get_data(selected_codes, train_combined, num_windows, window_size)
X_val, labels_val = get_data(selected_codes, val_combined, num_windows, window_size)
X_test, labels_test = get_data(selected_codes, test_combined, num_windows, window_size)
num_iters = max(X_train.shape[0]//batch_size,1)
num_features = 61

tf.reset_default_graph()

# Input and Target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.int32, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=70,
                                      max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

# Training
for e in range(2):
    for i in range(num_iters):
        start = i*batch_size
        end = (i+1)*batch_size
        batch_x = X_train[start:end]
        batch_y = labels_train[start:end]
        _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
        if i % 100 == 0:
            acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
            train_acc.append([i,acc])
            train_loss.append([i,l])
            #print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))


INFO:tensorflow:Constructing forest with params = 


INFO:tensorflow:Constructing forest with params = 


INFO:tensorflow:{'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'num_classes': 70, 'num_outputs': 1, 'num_features': 61, 'split_initializations_per_input': 1, 'min_split_samples': 5, 'base_random_seed': 0, 'bagged_num_features': 61, 'regression': False, 'num_trees': 70, 'num_output_columns': 71, 'max_fertile_nodes': 5000, 'split_after_samples': 250, 'dominate_fraction': 0.99, 'num_splits_to_consider': 61, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'max_nodes': 10000, 'bagged_features': None}


INFO:tensorflow:{'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'num_classes': 70, 'num_outputs': 1, 'num_features': 61, 'split_initializations_per_input': 1, 'min_split_samples': 5, 'base_random_seed': 0, 'bagged_num_features': 61, 'regression': False, 'num_trees': 70, 'num_output_columns': 71, 'max_fertile_nodes': 5000, 'split_after_samples': 250, 'dominate_fraction': 0.99, 'num_splits_to_consider': 61, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'max_nodes': 10000, 'bagged_features': None}


INFO:tensorflow:training graph for tree: 0


...

INFO:tensorflow:training graph for tree: 67


INFO:tensorflow:training graph for tree: 68


INFO:tensorflow:training graph for tree: 68


INFO:tensorflow:training graph for tree: 69


INFO:tensorflow:training graph for tree: 69


In [176]:
sess.run(accuracy_op, feed_dict={X: X_train, Y: labels_train})

0.78211933

In [177]:
sess.run(accuracy_op, feed_dict={X: X_val, Y: labels_val})

0.59786665

In [178]:
sess.run(accuracy_op, feed_dict={X: X_test, Y: labels_test})

0.65599996

In [32]:
print(len(X_test))

5400


In [123]:
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(X_train, labels_train)
print(model.feature_importances_)

[ 0.00273814  0.00253635  0.01515748  0.01927305  0.00182633  0.02553612
  0.0065513   0.00686261  0.00292177  0.00331091  0.00095981  0.00017534
  0.00252512  0.0027628   0.00341728  0.05772641  0.00179085  0.00502968
  0.0030682   0.0092896   0.00458009  0.00450392  0.00777067  0.00744098
  0.0040849   0.00424991  0.00456584  0.00308157  0.00586256  0.00468524
  0.00333083  0.0028294   0.00345417  0.00323812  0.04136428  0.04642128
  0.05048617  0.0469083   0.02860239  0.0166987   0.00236721  0.00518145
  0.0025864   0.00465047  0.00541778  0.00588157  0.00345407  0.00040316
  0.00643875  0.00895183  0.00804712  0.02782989  0.02076714  0.01540837
  0.03958593  0.03987956  0.06608332  0.06653438  0.06413819  0.06525026
  0.07352466]


In [67]:
np.argmax(model.feature_importances_)

57

In [85]:
model.fit(X_train, labels_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [71]:
np.argsort(model.feature_importances_)[:10]

array([11, 10, 16, 47,  4,  0,  8, 40, 12, 13])

In [124]:
model.score(X_test, labels_test)

0.19739130434782609

In [97]:
np.asarray(list(train_combined[514][42004][0][0]))[np.argsort(model.feature_importances_)[:30]]

array(['Dpf Regen Inhibit Sw', 'DTCID', 'Eng Coolant Level',
       'Cruise Status', 'Fan Speed', 'Clutch Switch',
       'Event Type Description', 'Brake Switch', 'Acc Pedal Position',
       'Dpf Thermal Mngmnt', 'Vehicle Speed', 'Part Trap Diff Press',
       'Cmd Eng Fuel Press', 'EngTurbo1Pos', 'Drvr Demand Torque',
       'EngTurbo1Boost', 'Eng Percent Torque', 'Engine Speed',
       'Eng Man Abs Pressure', 'Eng Demand Torque', 'Eng Egr Valve Pos',
       'Eng Air Flow Rate', 'EngInjRail1Press', 'EngTurbo1Speed',
       'Trip Run Time', 'Eng Fuel Del Press', 'Eng Oil Pressure',
       'Keyswitch Bat Pot', 'Eng Exhaust Gas Temp', 'Cat Intake Gas Temp'], 
      dtype='<U28')

In [80]:
np.asarray(list(train_combined[514][42004][0][0]))[57]

'Lifetime Idle Fuel'